In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)


import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
import pyminizip as pz

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']
municipios['regional'] = municipios['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ 'Regional'
municipios

In [ ]:
notifica = Notifica()
notifica.load('notifica',compress=False)

In [ ]:
notifica.df.dtypes

In [ ]:
notifica.df = notifica.df.loc[((notifica.df['classificacao_final']==2)&(notifica.df['excluir_ficha']==2)&(notifica.df['status_notificacao'].isin([1,2])))]

notifica.df = pd.merge(notifica.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','mun_resid','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica.df = pd.merge(notifica.df.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

notifica.df['exame_nome'] = notifica.toggle_merge_table('exame',inplace=False)
notifica.toggle_merge_table('sexo')

notifica.df = notifica.df.loc[((notifica.df['sexo']!='N')&(notifica.df['mun_resid'].notna())&(notifica.df['data_diagnostico'].notna())&(notifica.df['paciente'].str.len() > 5))]

In [ ]:
display(notifica.df[['id','tipo_paciente']].join(notifica.tables['tipo_paciente'].set_index('id'),on='tipo_paciente',rsuffix='_name').groupby(['tipo_paciente','tipo_paciente_name']).count())

In [ ]:
paciente_rua = notifica.df.loc[(notifica.df['tipo_paciente'] == 5) & (notifica.df['classificacao_final'] == 2) & (notifica.df['uf_residencia'] == 41)]
paciente_rua = paciente_rua.loc[(notifica.df['data_diagnostico'].dt.year >= 2021) | (notifica.df['data_cura_obito'].dt.year >= 2021)]
paciente_rua

In [ ]:
paciente_rua['regional'] = paciente_rua['rs'].apply(lambda x: str(int(x)).zfill(2)) +'ª '+ 'Regional'

paciente_rua_mun = paciente_rua.groupby(['regional','mun_resid','evolucao'])[['id']].count().unstack(-1).fillna(0).droplevel(0,1)
paciente_rua_mun['Total'] = paciente_rua_mun.sum(1)
paciente_rua_mun.loc[('','Total'),:] = paciente_rua_mun.sum(0)
paciente_rua_mun.astype(int)

In [ ]:
paciente_rua_mun.to_excel(join(default_output,'paciente_rua_2021.xlsx'))

In [ ]:
paciente_rua_mun.tail(10)